<a href="https://colab.research.google.com/github/agilan123/PlannerAI/blob/main/AI_Scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import sys
import json



df = pd.read_csv('Shuffled_ML_training_scheduler.csv')
queries = df['query'].values
priorities = df['priority'].values

X_train, X_test, y_train, y_test = train_test_split(queries, priorities, test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = max(len(x) for x in X_train_seq)
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

model = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


history = model.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))


test_loss, test_acc = model.evaluate(X_test_padded, y_test, verbose=2)
print('\nTest accuracy:', test_acc)



predictions = model.predict(X_test_padded)
predicted_priorities = np.argmax(predictions, axis=1)


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 22, 16)            160000    
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
 1 (GlobalAveragePooling1D)                                      
                                                                 
 dense_34 (Dense)            (None, 24)                408       
                                                                 
 dense_35 (Dense)            (None, 4)                 100       
                                                                 
Total params: 160508 (626.98 KB)
Trainable params: 160508 (626.98 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
175/175 [==============================] - 1s 4ms/step - loss: 1.208

In [ ]:


new_query = ["I have a doctors appointment sometime this week"]
sequence = tokenizer.texts_to_sequences(new_query)


padded_sequence = pad_sequences(sequence, maxlen=100, padding='post')  # Use the same maxlen as during training


prediction = model.predict(padded_sequence)

predicted_priority = np.argmax(prediction, axis=1)
print(f"Predicted Priority: {predicted_priority[0]}")


1/1 [==============================] - 0s 15ms/step
Predicted Priority: 2


In [ ]:


queries = df['query'].values
intensities = df['intensity'].values  # Assuming 'intensity' column exists

# Split the data
X_train, X_test, y_train, y_test = train_test_split(queries, intensities, test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


max_length = max(len(x) for x in X_train_seq)  # Or set a predefined max_length
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Adjust the model to fit the intensity prediction
# The last Dense layer might need adjustments based on the range of your intensity values
model = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='linear')  # For intensity, you might use a single neuron with linear activation
])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])  # Use MSE and MAE for regression tasks
model.summary()

# Train the model
history = model.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test_padded, y_test, verbose=2)
print('\nTest MAE:', test_mae)

# Making predictions
predictions = model.predict(X_test_padded)
# For intensity, you might directly use the predictions without needing to apply argmax since it's a regression task now


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 22, 16)            160000    
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
 2 (GlobalAveragePooling1D)                                      
                                                                 
 dense_36 (Dense)            (None, 24)                408       
                                                                 
 dense_37 (Dense)            (None, 1)                 25        
                                                                 
Total params: 160433 (626.69 KB)
Trainable params: 160433 (626.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
175/175 [==============================] - 1s 3ms/step - loss: 3.705

In [ ]:
# Assuming `tokenizer` and `model` are already defined and properly trained

# Example query
query = "I have a doctors appointment"

# Tokenize and pad the query
sequence = tokenizer.texts_to_sequences([query])
padded_sequence = pad_sequences(sequence, maxlen=100, padding='post')  # Ensure maxlen matches training setup

# Predict intensity
predicted_intensity = model.predict(padded_sequence)

# Assuming intensity is a single regression output from the model
print(f"Predicted Intensity: {predicted_intensity[0][0]}")


1/1 [==============================] - 0s 16ms/step
Predicted Intensity: 1.8994007110595703
